In [1]:
import numpy as np
import glob
from matplotlib import pyplot as plt
import os
from sklearn import cross_validation
import pandas as pd
import cv2
import datetime as dt

# replace xgboost with lightgbm
from lightgbm.sklearn import LGBMRegressor

C:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Tips on using pre-trained model in Keras
# http://stackoverflow.com/questions/41764041/fine-tuning-pretrained-model-in-keras/41791568
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

# Furter import to allow for re-training / fine-tuning
from keras.models import Model
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


Set path variables:

In [3]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

train_path = "C:/deep_learning/kaggle/amazon/train/"
test_path = "C:/deep_learning/kaggle/amazon/test/"

#Location for saving serialized objects
# obj_save_path = "/home/alex/Desktop/Rainforest/Models/XGB/Objects/"
feat_output_path = ""
# subm_output_path = "/home/alex/Desktop/Rainforest/Submissions/"

train = pd.read_csv("C:/deep_learning/kaggle/amazon/train_v2.csv")
test =  pd.read_csv("C:/deep_learning/kaggle/amazon/sample_submission_v2.csv")
train.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [4]:
files = os.listdir(train_path)
imgs = [img_to_array(load_img(train_path + p, target_size=[224,224])) for p in files]
im = np.array(imgs)
print(im.shape)

(103, 224, 224, 3)


In [4]:
im = img_to_array(load_img(train_path + 'train_0.jpg', target_size=[224,224]))
im = np.expand_dims(im, axis=0)
print(im.shape)

(1, 224, 224, 3)


https://github.com/fchollet/keras/blob/master/keras/applications/resnet50.py
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 224)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 197.
            E.g. `(200, 200, 3)` would be one valid value.

In [5]:
print('Defining Model', dt.datetime.now())
base_model = ResNet50(weights='imagenet', include_top=False) # peel off the last layer to get the features
print('Model Defined', dt.datetime.now())
print(base_model.summary())

Defining Model 2017-05-24 22:54:06.279791
Model Defined 2017-05-24 22:54:41.321960
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, None, None, 3) 0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, None, None, 64 256         conv1[0][0]                      
________

In [6]:
print('Extracting Features with 100 images', dt.datetime.now())
feats=base_model.predict(im)
print(feats)
print(feats.shape)
print('Finished Extracting Features with 100 images', dt.datetime.now())

Extracting Features with 100 images 2017-05-24 22:54:41.679291
[[[[ 0.01727332  0.          0.         ...,  9.19477272  0.          0.06318483]]]]
(1, 1, 1, 2048)
Finished Extracting Features with 100 images 2017-05-24 22:54:45.085375
